In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset from a CSV file
df = pd.read_csv('/content/human_annotations_same - human_annotations_same.csv')

# Preprocess the text data
texts = df['text']
labels = df['human_label1']

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad the sequences to ensure uniform length
max_sequence_length = max(len(sequence) for sequence in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)



# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Build the RNN model
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for the padding token
embedding_dim = 50  # Dimensionality of the word embeddings
hidden_units = 8  # Number of LSTM units
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=hidden_units))
model.add(Dense(units=1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32)

# Make predictions on the testing set
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Epoch 1/5
169/169 [==============================] - 507s 3s/step - loss: 0.5956 - accuracy: 0.6625
Epoch 2/5
169/169 [==============================] - 502s 3s/step - loss: 0.4832 - accuracy: 0.7749
Epoch 3/5
169/169 [==============================] - 496s 3s/step - loss: 0.3995 - accuracy: 0.8325
Epoch 4/5
169/169 [==============================] - 493s 3s/step - loss: 0.3373 - accuracy: 0.8639
Epoch 5/5
43/43 [==============================] - 12s 246ms/step
Accuracy: 0.7823179791976226


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.78      0.74       850
           1       0.54      0.45      0.49       496

    accuracy                           0.66      1346
   macro avg       0.63      0.62      0.62      1346
weighted avg       0.65      0.66      0.65      1346

